In [1]:
#
# user settings
#
input_xml_file = 'test.xml.gz'
output_csv_file = 'tracks.csv'

In [4]:
#
# load useful libraries
#
import json
import xmltodict
import pandas as pd
import numpy as np
import gzip
#from neo4j import GraphDatabase

In [7]:
with gzip.open(input_xml_file, 'rb') as f:
    #file_content = f.read()
    data_dict = xmltodict.parse(f.read())

In [10]:
#
# iterate through the tracks
#
track_item_list = []
for track in data_dict['DJ_PLAYLISTS']['COLLECTION']['TRACK']:

    #
    # retrieve basic track information
    #
    track_dict_original = {
        'name' : track['@Name'].strip(),
        'artist' : track['@Artist'].strip(),
        'album' : track['@Album'].strip(),
        'year' : track['@Year'].strip(),
        'genre' : track['@Genre'].strip().lower(),
        'tonality' : track['@Tonality'].strip(),
        'bpm_average' : track['@AverageBpm'].strip(),
    }

    #
    # non-numeric keys
    #
    try:
        if not track_dict_original['tonality'][0].isdigit():
            track_dict_original['tonality'] = None
    except:
        track_dict_original['tonality'] = None
    
    #
    # extract energy level
    #
    comments = track['@Comments'].strip()
    try:
        track_dict_original['energy'] = np.int64(
            comments
            .lower()
            .split('energy ')[-1]
            .split(' ')[0]
        )
    except:
        track_dict_original['energy'] = None

    #
    # Bpm
    #
    track_dict_original['bpm_average'] = np.float64(track_dict_original['bpm_average'])
        
    #
    # deal with NA values
    #
    track_dict = {}
    for key in track_dict_original:
        if track_dict_original[key] == '':
            track_dict[key] = None
        else:
            track_dict[key] = track_dict_original[key]
    del(track_dict_original)

    #
    # append
    #
    track_item_list.append(track_dict)


In [13]:
#
# assemble data frame
#
df = pd.DataFrame(track_item_list).dropna().reset_index().drop(columns = ['index'])
df['song_id'] = df.index

df['year'] = [int(x) for x in df['year']]
df['energy'] = [int(x) for x in df['energy']]



In [16]:
df.head()

,name,artist,album,year,genre,tonality,bpm_average,energy,song_id
0,"Sine From Above (Chester Lockhart, Mood Killer...",Lady Gaga,Dawn Of Chromatica,2021,edm,5A,174.92,7,0
1,Let the Bass Kick In Miami Bitch,Chuckie & LMFAO,100% Clubland EDM Bangers,0,edm,8A,128.11,7,1
2,Sorry For Party Rocking,LMFAO,Clubland 21 (CD1),2012,dance,6A,134.00,7,2
3,On My Way to Hell,Połoz & Tinnitus,"Cyberpunk 2077: Radio, Vol. 2 (Original Soundt...",2020,soundtracks,9A,137.03,5,3
4,Kooler Than Jesus (Electric Messiah Mix),My Life With The Thrill Kill Kult,Confessions Of A Knife,1990,electronica,11A,124.67,7,4


In [19]:
df_artist = pd.DataFrame(
    {
        'artist_name_pre_qc' : df['artist'].unique(),
    }
)

df_artist['artist_name'] = [x.strip() for x in df_artist['artist_name_pre_qc']]

df_artist.reset_index(inplace = True)
df_artist.rename(columns = {'index' : 'artist_id'}, inplace = True)

In [21]:
df_artist

,artist_id,artist_name_pre_qc,artist_name
0,0,Lady Gaga,Lady Gaga
1,1,Chuckie & LMFAO,Chuckie & LMFAO
2,2,LMFAO,LMFAO
3,3,Połoz & Tinnitus,Połoz & Tinnitus
4,4,My Life With The Thrill Kill Kult,My Life With The Thrill Kill Kult
...,...,...,...
1822,1822,KXD-LvL,KXD-LvL
1823,1823,Alex Farell,Alex Farell
1824,1824,Divina Blasfemia,Divina Blasfemia
1825,1825,Lethargy,Lethargy


In [ ]:
df_album = pd.DataFrame({'album_name' : df['album'], 'artist_name' : df['artist_name']})

In [ ]:
#
# save
#
df.to_csv(output_csv_file, index=False)